In [1]:
from google.colab import drive
import sys
drive.mount('/content/gdrive')
sys.path.append('/content/gdrive/MyDrive/AIM')
!ls -l /content/gdrive/MyDrive/AIM/*.pkl
import aim_util

Mounted at /content/gdrive
-rw------- 1 root root 18958 May 28 18:39 /content/gdrive/MyDrive/AIM/citizen_1864_xform0_carlv_bbox.pkl
-rw------- 1 root root 18958 May 30 13:37 /content/gdrive/MyDrive/AIM/citizen_1864_xform0_carlv_raw_bbox.pkl
-rw------- 1 root root 17591 May 30 13:37 /content/gdrive/MyDrive/AIM/citizen_1864_xform0_carlv_sort_dist_bbox.pkl
-rw------- 1 root root 19113 May 24 19:19 /content/gdrive/MyDrive/AIM/citizen_1864_xform1_carlv_bbox.pkl
-rw------- 1 root root 19113 May 30 14:26 /content/gdrive/MyDrive/AIM/citizen_1864_xform1_carlv_raw_bbox.pkl
-rw------- 1 root root 17769 May 30 14:26 /content/gdrive/MyDrive/AIM/citizen_1864_xform1_carlv_sort_dist_bbox.pkl
-rw------- 1 root root 15241 May 24 19:23 /content/gdrive/MyDrive/AIM/citizen_1864_xform2_carlv_bbox.pkl
-rw------- 1 root root 16344 May 30 19:47 /content/gdrive/MyDrive/AIM/citizen_1864_xform2_carlv_raw_bbox.pkl
-rw------- 1 root root 15552 May 30 19:47 /content/gdrive/MyDrive/AIM/citizen_1864_xform2_carlv_sort_

In [3]:
gdrive_path = "/content/gdrive/MyDrive/AIM/"
image_path = '/content/gdrive/MyDrive/AIM/citizen_1864_rescan.jpg'
pred_df1_path = '/content/gdrive/MyDrive/AIM/citizen_1864_xform0_carlv_sort_dist_bbox.pkl'
pred_df2_path = '/content/gdrive/MyDrive/AIM/citizen_1864_xform1_carlv_sort_dist_bbox.pkl'


In [6]:
import pickle
def read_from_pickle(path):
    """Read the Pickle file into a Pandas dataframe"""
    with open(path, 'rb') as f:
        print(f"Dataframe loading from: {path}")
        return pickle.load(f)

In [16]:
# Model A
from decimal import Decimal
import pandas as pd

def merge_pd(df1, df2):
    # Initializing df_merge with the same columns as df1 and adding a 'color' column
    df_merge = pd.DataFrame(columns=df1.columns.tolist() + ['color'])

    # Iterating through each row of df1 and df2
    for index1, row1 in df1.iterrows():
        for index2, row2 in df2.iterrows():

            # Removing the decimal portion of the data values but not the strings
            row1_updated = row1.apply(lambda x: x if isinstance(x, str) else int(Decimal(x).quantize(Decimal('1'))))
            row2_updated = row2.apply(lambda x: x if isinstance(x, str) else int(Decimal(x).quantize(Decimal('1'))))

            print(f"row1 word {row1_updated['word']}, row2 word {row2_updated['word']}")
            # Checking if the 'word' in df1 and df2 matches
            if row1_updated['word'] == row2_updated['word']:
                # Append df1 column to df_merge and mark the 'color' as (0, 255, 0)
                temp_df = row1_updated.to_frame().T
                temp_df['color'] = "GREEN"
                df_merge = df_merge._append(temp_df, ignore_index=True)
            else:
                # Append df1 column to df_merge and mark the 'color' as (0, 0, 255)
                temp_df = row1_updated.to_frame().T
                temp_df['color'] = "BLUE"
                df_merge = df_merge._append(temp_df, ignore_index=True)

                # Append df2 column to df_merge and mark the 'color' as (255, 0, 0)
                temp_df = row2_updated.to_frame().T
                temp_df['color'] = "RED"
                df_merge = df_merge._append(temp_df, ignore_index=True)

    return df_merge

#Example to test the merge method
df1 = pd.DataFrame({'word': ['united'], 'CENTER_X': [890.572021], 'CENTER_Y': [262.360382],
                   'DIST_ORIGEN': [928.413452], 'DIST_Y': [262.360382],
                   'UL_X': [628.225586], 'UL_Y': [214.346313],
                   'LR_X': [1152.918457], 'LR_Y': [310.374451] })

df2 = pd.DataFrame({'word': ['united'], 'CENTER_X': [891], 'CENTER_Y': [263],
                    'DIST_ORIGEN': [929], 'DIST_Y': [261],
                    'UL_X': [629], 'UL_Y': [215],
                    'LR_X': [1153],'LR_Y': [311] })

result = merge_pd(df1, df2)
print(result)



     word CENTER_X CENTER_Y DIST_ORIGEN DIST_Y UL_X UL_Y  LR_X LR_Y  color
0  united      891      262         928    262  628  214  1153  310  GREEN


In [17]:
df1 = read_from_pickle(pred_df1_path)
df2 = read_from_pickle(pred_df2_path)
df_merge = merge_pd(df1, df2)
# print(df_merge)

KeyboardInterrupt: 

In [ ]:
merge_path = '/content/gdrive/MyDrive/AIM/citizen_1864_xform_carlv_merge_1_2_bbox.pkl'
aimutil.write_to_pickle(merge_path)

In [ ]:
# Model B

In [10]:


df1 = read_from_pickle(pred_df1_path)
df2 = read_from_pickle(pred_df2_path)
df_merge = merge_pd(df1, df2)
print(df_merge)

DF1:
     word  CENTER_X  CENTER_Y  DIST_ORIGEN  DIST_Y    UL_X    UL_Y     LR_X  \
0  united    890.57    262.36       928.41  262.36  628.23  214.35  1152.92   

     LR_Y  
0  310.37  

DF2:
     word  CENTER_X  CENTER_Y  DIST_ORIGEN  DIST_Y    UL_X    UL_Y    LR_X  \
0  states    400.57    500.36       300.41  800.36  280.23  414.35  552.92   

     LR_Y  
0  610.37  


ValueError: invalid literal for int() with base 10: 'united'